**Принцип работы SMT**

**SMT** (*Statistical Machine Translation = статистический машинный перевод*)

*Определение*: машинный перевод, в основе которого лежит поиск наиболее **правдоподобный** вариант перевода предложения/текста с исходного языка (*x*) на язык перевода (*y*) на основе сведений о частоте совместной встречаемости слов друг с другом.

*Компоненты модели SMT*:

1.   **N-граммная языковая модель** - создаётся модель представляющая то, насколько вероятно встречаются вместе конструкции на уровне слов и фраз в целевом языке *y*;
2.   **t-model** (*модель перевода*) - собирается статистическая информация о соответствии фраз на языке *x* со словами на языке *y* и создаётся модель на основе теории вероятности;
3.   **Декодер** - выбираются наиболее правдоподобных (лексически и грамматически) результаты, из них выбирается один в качестве перевода, который и выводится.




_____________________________________________________________________

**Повторение эксперимента**

In [1]:
import tarfile

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

In [2]:
with tarfile.open('de-en.tgz', 'r:gz') as tar:
  tar.extractall()

In [3]:
!ls

de-en.de  de-en.en  de-en.tgz  sample_data


In [4]:
with open('de-en.de', 'r') as f:
  german = f.read().split('\n')[:-1]

with open('de-en.en', 'r') as f:
  english = f.read().split('\n')[:-1]

print("Данные языка X:\n", german)
print("Данные языка Y:\n", english)

Данные языка X:
 ['Wiederaufnahme der Sitzungsperiode', 'Ich erkläre die am Freitag , dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen , wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe , daß Sie schöne Ferien hatten .', 'Wie Sie feststellen konnten , ist der gefürchtete " Millenium-Bug " nicht eingetreten . Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden .', 'Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen .', 'Heute möchte ich Sie bitten - das ist auch der Wunsch einiger Kolleginnen und Kollegen - , allen Opfern der Stürme , insbesondere in den verschiedenen Ländern der Europäischen Union , in einer Schweigeminute zu gedenken .', 'Ich bitte Sie , sich zu einer Schweigeminute zu erheben .', '( Das Parlament erhebt sich zu einer Schweigeminute . )', 'Frau Präsidentin , zur Geschäftsordnung .', 'Wie Sie sicher aus 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(english, german)

print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

> Обучающая выборка:

Текст на немецком: Ich halte das für besser als jetzt lange Stimmerklärungen abzugeben .
 Его перевод на английский: I think this is a better solution than proceeding now to extremely time-consuming explanations of votes .


Текст на немецком: Ich meine damit , daß wir uns am Anfang des Planungszeitraums 2000-2006 befinden , der angesichts von zwei großen Herausforderungen keineswegs als Routineangelegenheit aufgefaßt werden darf .
 Его перевод на английский: In brief , I would like to say that we are entering the period when we are called upon to manage the programming for 2000-2006 , which must be no routine period for the good reason that we have two major challenges to face .


Текст на немецком: Bestimmte europäische Industriezweige wie der Schiffbau , die Luftfahrt- und die Stahlindustrie sind bereits Opfer wettbewerbspolitischer Maßnahmen geworden und haben beträchtliche Verluste hinnehmen müssen .
 Его перевод на английский: Certain sectors of European ind

In [6]:
def tokenize(sentences):
  # функция возвращает списки слов
  return [sentence.split() for sentence in sentences]

# токенизируем каждую выборку
X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens)

Образец токенизированного текста: [['I', 'think', 'this', 'is', 'a', 'better', 'solution', 'than', 'proceeding', 'now', 'to', 'extremely', 'time-consuming', 'explanations', 'of', 'votes', '.'], ['In', 'brief', ',', 'I', 'would', 'like', 'to', 'say', 'that', 'we', 'are', 'entering', 'the', 'period', 'when', 'we', 'are', 'called', 'upon', 'to', 'manage', 'the', 'programming', 'for', '2000-2006', ',', 'which', 'must', 'be', 'no', 'routine', 'period', 'for', 'the', 'good', 'reason', 'that', 'we', 'have', 'two', 'major', 'challenges', 'to', 'face', '.'], ['Certain', 'sectors', 'of', 'European', 'industry', ',', 'such', 'as', 'the', 'shipbuilding', 'industry', ',', 'air', 'transport', 'and', 'the', 'steel', 'industry', ',', 'which', 'have', 'been', 'hard', 'hit', 'by', 'existing', 'competition', 'policy', ',', 'have', 'suffered', 'tremendously', '.'], ['The', 'questions', 'answered', 'previously', 'referred', 'to', 'Mrs', 'de', 'Palacio', "'", 's', 'intervention', ',', 'on', 'another', 'occa

In [7]:
x_vocab = Counter(' '.join(german).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь немецких словоформ: {x_vocab}\n Всего {len(x_vocab)} словоформ")
print(f"\nCловарь английских словоформ: {y_vocab}\n Всего {len(y_vocab)} словоформ")

Словарь немецких словоформ: dict_keys(['Wiederaufnahme', 'der', 'Sitzungsperiode', 'Ich', 'erkläre', 'die', 'am', 'Freitag', ',', 'dem', '17.', 'Dezember', 'unterbrochene', 'des', 'Europäischen', 'Parlaments', 'für', 'wiederaufgenommen', 'wünsche', 'Ihnen', 'nochmals', 'alles', 'Gute', 'zum', 'Jahreswechsel', 'und', 'hoffe', 'daß', 'Sie', 'schöne', 'Ferien', 'hatten', '.', 'Wie', 'feststellen', 'konnten', 'ist', 'gefürchtete', '"', 'Millenium-Bug', 'nicht', 'eingetreten', 'Doch', 'sind', 'Bürger', 'einiger', 'unserer', 'Mitgliedstaaten', 'Opfer', 'von', 'schrecklichen', 'Naturkatastrophen', 'geworden', 'Im', 'Parlament', 'besteht', 'Wunsch', 'nach', 'einer', 'Aussprache', 'im', 'Verlauf', 'dieser', 'in', 'den', 'nächsten', 'Tagen', 'Heute', 'möchte', 'ich', 'bitten', '-', 'das', 'auch', 'Kolleginnen', 'Kollegen', 'allen', 'Opfern', 'Stürme', 'insbesondere', 'verschiedenen', 'Ländern', 'Union', 'Schweigeminute', 'zu', 'gedenken', 'bitte', 'sich', 'erheben', '(', 'Das', 'erhebt', ')', 'F

In [8]:
uniform = 1 / (len(x_vocab) * len(y_vocab))

round(uniform, 3)

0.0

In [12]:
t = {}

for i in range(len(X_train)):
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      t[(word_x, word_y)] = uniform

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | sixth   ->   recht | Вероятность: 0.0
Соответствие | sixth   ->   oft | Вероятность: 0.0
Соответствие | sixth   ->   nicht | Вероятность: 0.0
Соответствие | sixth   ->   gewünschten | Вероятность: 0.0
Соответствие | sixth   ->   Erfolg | Вероятность: 0.0
Соответствие | sixth   ->   zeitigen | Вероятность: 0.0
Соответствие | periodic   ->   Aus | Вероятность: 0.0
Соответствие | periodic   ->   kann | Вероятность: 0.0
Соответствие | periodic   ->   man | Вероятность: 0.0
Соответствие | periodic   ->   vorsichtig | Вероятность: 0.0
Соответствие | periodic   ->   schlußfolgern | Вероятность: 0.0
Соответствие | periodic   ->   Anreize | Вероятность: 0.0
Соответствие | periodic   ->   recht | Вероятность: 0.0
Соответствие | periodic   ->   oft | Вероятность: 0.0
Соответствие | periodic   ->   nicht | Вероятность: 0.0
Соответствие | periodic   ->   gewünschten | Вероятность: 0.0
Соответствие | periodic   ->   E

In [13]:
epochs = 7

In [14]:
for epoch in range(epochs):

  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        count[(word_x, word_y)] = 0
        total[word_y] = 0

  for i in range(len(X_train)):
    total_stat = {}

    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0
      for word_y in y_train_tokens[i]:
        total_stat[word_x] += t[(word_x, word_y)]

    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  for i in range(len(X_train)):
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | sixth   ->   recht | Вероятность: 0.0
Соответствие | sixth   ->   oft | Вероятность: 0.0
Соответствие | sixth   ->   nicht | Вероятность: 0.0
Соответствие | sixth   ->   gewünschten | Вероятность: 0.001
Соответствие | sixth   ->   Erfolg | Вероятность: 0.0
Соответствие | sixth   ->   zeitigen | Вероятность: 0.01
Соответствие | periodic   ->   Aus | Вероятность: 0.0
Соответствие | periodic   ->   kann | Вероятность: 0.0
Соответствие | periodic   ->   man | Вероятность: 0.0
Соответствие | periodic   ->   vorsichtig | Вероятность: 0.01
Соответствие | periodic   ->   schlußfolgern | Вероятность: 0.01
Соответствие | periodic   ->   Anreize | Вероятность: 0.01
Соответствие | periodic   ->   recht | Вероятность: 0.0
Соответствие | periodic   ->   oft | Вероятность: 0.0
Соответствие | periodic   ->   nicht | Вероятность: 0.0
Соответствие | periodic   ->   gewünschten | Вероятность: 0.001
Соответствие | periodic 

In [15]:
tokens = ' '.join(german).split()

bigram_model = defaultdict(list)

for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(bigram_model)

def decoder(model, steps=5):
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

defaultdict(<class 'list'>, {'Wiederaufnahme': ['der', 'der'], 'der': ['Sitzungsperiode', 'gefürchtete', 'Wunsch', 'Wunsch', 'Stürme', 'Europäischen', 'Presse', 'dem', 'Präsidentin', 'Unzulässigkeit', 'in', 'relativen', 'Grundsatz', 'relativen', 'gemeinsamen', 'Aussprache', 'vom', 'gesamten', 'Institution', 'zur', 'Vollstreckung', 'Position', 'Fall', 'Zugang', 'Staatsanwalt', 'letzten', 'Barentsee', 'Ihnen', 'Beschlüsse', 'ersten', 'Tagesordnung', 'Quästoren', 'Arbeitsschutzausschuß', 'Vizepräsidentin', 'Meinung', 'Fall', 'Tageszeitung', 'heutigen', 'eine', 'Vergangenheit', 'dortigen', 'großen', 'EU-Mitgliedstaaten', 'EU', 'Tagesordnung', 'Tagesordnung', 'Geschäftsordnung', 'Konferenz', 'Präsidenten', 'Kommission', 'Kommission', 'Kommission', 'Fraktion', 'Sozialdemokratischen', 'die', 'Konferenz', 'Präsidenten', 'seine', 'vorangegangenen', 'empfohlen', 'kürzestmöglichen', 'vergangenen', 'Konferenz', 'Präsidenten', 'Lage', 'Lage', 'Abstimmung', 'Kommission', 'Lage', 'Meinung', 'Lage', '

In [30]:
sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def translate(token):
  for token in sentence:
    if len(token) > 0:
      for element in sorted_t:
        if element[0][1] == token:
          return element[0][0]

for sentence in y_test_tokens:
  print("Оригинальное предложение:", ' '.join(sentence))
  translation = []
  for token in sentence:
    translation.append(translate(token))
  print("Перевод:", ' '.join(translation))

Оригинальное предложение: Den Änderungsanträgen 6 und 7 des Berichterstatters sollte deshalb zugestimmt werden .
Перевод: would would would would would would would would would would would would
Оригинальное предложение: Aber ich glaube , daß wir alles unternehmen sollten , um den Transport gefährlicher Güter so gering wie möglich zu halten , und zwar in allen Ländern , ob Transitländer oder nicht .
Перевод: however however however however however however however however however however however however however however however however however however however however however however however however however however however however however however however however however
Оригинальное предложение: Ich schlage vor , daß wir über den Antrag der Sozialdemokratischen Fraktion , die Erklärung der Kommission über ihre strategischen Ziele wieder auf die Tagesordnung zu setzen , abstimmen .
Перевод: I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I
Оригинальное предложение: Dem kann ich zu

In [36]:
from nltk.translate.bleu_score import corpus_bleu

reference = [[X_test_tokens[i]] for i in range(len(X_test_tokens))]
candidate = [translate(token) for token in y_test_tokens]

bleu_score = corpus_bleu(reference, [candidate[i] for i in range(len(candidate))])

print("BLEU Score:", bleu_score)

BLEU Score: 0


In [37]:
reference

[[['The',
   'rapporteur',
   "'",
   's',
   'Amendments',
   'Nos',
   '6',
   'and',
   '7',
   ',',
   'therefore',
   ',',
   'deserve',
   'our',
   'support',
   '.']],
 [['But',
   'I',
   'believe',
   'that',
   'we',
   'should',
   'do',
   'all',
   'we',
   'can',
   'to',
   'keep',
   'the',
   'transport',
   'of',
   'dangerous',
   'goods',
   'to',
   'a',
   'minimum',
   ',',
   'in',
   'all',
   'countries',
   ',',
   'whether',
   'they',
   'are',
   'transit',
   'countries',
   'or',
   'not',
   '.']],
 [['I',
   'propose',
   'that',
   'we',
   'vote',
   'on',
   'the',
   'request',
   'of',
   'the',
   'Group',
   'of',
   'the',
   'Party',
   'of',
   'European',
   'Socialists',
   'that',
   'the',
   'Commission',
   'statement',
   'on',
   'its',
   'strategic',
   'objectives',
   'should',
   'be',
   'reinstated',
   '.']],
 [['Personally',
   ',',
   'I',
   'at',
   'least',
   'am',
   'totally',
   'in',
   'favour',
   'of',
   'the',


In [38]:
candidate

['there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
 'there',
